In [1]:

import numpy as np
import seaborn as sns
import pandas as pd
import statsmodels.api as sm

<font size='5' face='Times new Roman'>
De US Factors heten: Anomalies_US  
<br>
De Global Factors heten: Anomalies_GF
<br>
De table 1: Anomaliescombinedtable
    <font>

<font size='4' face='Times new Roman'>
    Dit worden de US Factors
    <font>

In [265]:
file_url = 'https://raw.githubusercontent.com/Sebasleen/Seminargroup/Seminar/US%20Factors/anomalies.dta'

# Read the Stata file content
Anomalies = pd.read_stata(file_url)

# Display unique values in the 'anomaly' column
print(Anomalies['anomaly'].unique())

#Hier heb ik de global factors verwijderd
column_name = 'anomaly'
values_to_dropUS = ['glbab', 'glcma', 'glhml', 'glqmj', 'glrmw', 'glsmb', 'glumd']
ElementsUS = Anomalies[column_name].isin(values_to_dropUS)
Anomalies_US = Anomalies[~ElementsUS]

column_name = 'anomaly'
values_to_dropGF = ['ac', 'bab', 'cfp', 'cma', 'ep', 'hml', 'liq', 'ltrev', 'nsi', 'qmj', 'rmw', 'rvar',
'smb', 'strev', 'umd']
ElementsGF = Anomalies[column_name].isin(values_to_dropGF)
Anomalies_GF = Anomalies[~ElementsGF]

#Dit zijn dus alle 15 factors die alleen bij de US horen. 
print(Anomalies_US)

#En dit zijn alle 7 global factors
print(Anomalies_GF)



['ac' 'bab' 'cfp' 'cma' 'ep' 'hml' 'liq' 'ltrev' 'nsi' 'qmj' 'rmw' 'rvar'
 'smb' 'strev' 'umd' 'glbab' 'glcma' 'glhml' 'glqmj' 'glrmw' 'glsmb'
 'glumd']
       year  month anomaly    ret   time  global
0      1963      7      ac  2.170   42.0     0.0
1      1963      8      ac -0.197   43.0     0.0
2      1963      9      ac  0.600   44.0     0.0
3      1963     10      ac  6.463   45.0     0.0
4      1963     11      ac -2.260   46.0     0.0
...     ...    ...     ...    ...    ...     ...
10111  2019      8     umd  7.600  715.0     0.0
10112  2019      9     umd -6.850  716.0     0.0
10113  2019     10     umd  0.240  717.0     0.0
10114  2019     11     umd -2.620  718.0     0.0
10115  2019     12     umd -2.130  719.0     0.0

[10116 rows x 6 columns]
       year  month anomaly       ret   time  global
10116  1987      2   glbab  2.236918  325.0     1.0
10117  1987      3   glbab  1.828450  326.0     1.0
10118  1987      4   glbab -5.521739  327.0     1.0
10119  1987      5   glba

<font size='4' face='Times new Roman'>    

Voor de duidelijkheid, dit zijn de US factors:

- **-ac**: Accruals
- **-bab**: Betting against Beta
- **-cfp**: Cash-flow to price
- **-cma**: Investment (Conservative minus aggressive)
- **-ep**: Earnings to price
- **-hml**: Value
- **-liq**: Liquidity
- **-ltrev**: Long-term reversal
- **-nsi**: Net share issue
- **-qmj**: Quality minus junk
- **-rmw**: Profitability
- **-rvar**: Residual Variance
- **-smb**: Size
- **-strev**: Short-term reversals
- **-umd**: Momentum

En dit zijn de global factors die ik zojuist heb gedropt:

- **glbab**: Betting against beta
- **glcma**: Investment (Conservative minus aggressive)
- **glhml**: Value (high minus low)
- **glqmj**: Quality minus junk
- **glrmw**: Profitability (robust minus weak)
- **glumd**: Momentum (up minus down)
- **glsmb**: Size (small minus big)

</font>


In [228]:
#Table 1: Descriptive Statistics US Factors

AnomaliesUS = Anomalies_US.groupby(['anomaly']).agg({'ret': ['mean', 'std', 'count']}).reset_index()
AnomaliesUS = Anomalies_US.pivot_table(index='anomaly', values='ret', aggfunc=['mean', 'std', 'count'])
AnomaliesUS.columns = ['Mean', 'SD', 'ret_number']
AnomaliesUS.reset_index(inplace=True)

#Calculate the table stuff
AnomaliesUS.columns = ['anomaly', 'Mean', 'SD', 'ret_number']

# Calculate additional statistics
AnomaliesUS['ret_semean'] = AnomaliesUS['SD'] / np.sqrt(AnomaliesUS['ret_number'])

#Times 12 to annualize it
AnomaliesUS['ret'] = AnomaliesUS['Mean'] * 12 

#Times Squareroot 12 to annualize it
AnomaliesUS['sd'] = AnomaliesUS['SD'] * np.sqrt(12) 

#Calculate the t-stat by dividing the return by the standard error of the mean. Divide by 12 to
#annualize it
AnomaliesUS['tstat'] = AnomaliesUS['ret'] / AnomaliesUS['ret_semean'] /12

#Format Table
AnomaliesUS[['Mean']] = AnomaliesUS[['ret']].apply(lambda x: x.map("{:.1f}%".format))
AnomaliesUS[['SD']] = AnomaliesUS[['sd']].apply(lambda x: x.map("{:.1f}%".format))
AnomaliesUS[['T-value']] = AnomaliesUS[['tstat']].apply(lambda x: x.map("{:.2f}".format))

print(AnomaliesUS[['anomaly','Mean','SD','T-value']])


   anomaly  Mean     SD T-value
0       ac  2.8%   6.6%    3.19
1      bab  9.8%  11.2%    6.55
2      cfp  3.4%   8.6%    2.94
3      cma  3.3%   6.9%    3.59
4       ep  3.5%   8.9%    2.95
5      hml  3.6%   9.7%    2.82
6      liq  4.4%  11.6%    2.77
7    ltrev  2.5%   8.7%    2.16
8      nsi  2.8%   8.2%    2.52
9      qmj  4.6%   7.7%    4.47
10     rmw  3.1%   7.5%    3.13
11    rvar  1.6%  17.3%    0.68
12     smb  2.7%  10.4%    1.97
13   strev  6.0%  10.6%    4.21
14     umd  7.8%  14.5%    4.02


In [240]:
#Table 1: Descriptive Statistics US Factors

AnomaliesGF = Anomalies_GF.groupby(['anomaly']).agg({'ret': ['mean', 'std', 'count']}).reset_index()
AnomaliesGF = Anomalies_GF.pivot_table(index='anomaly', values='ret', aggfunc=['mean', 'std', 'count'])
AnomaliesGF.columns = ['Mean', 'SD', 'ret_number']
AnomaliesGF.reset_index(inplace=True)

#Calculate the table stuff
AnomaliesGF.columns = ['anomaly', 'Mean', 'SD', 'ret_number']

# Calculate additional statistics
AnomaliesGF['ret_semean'] = AnomaliesGF['SD'] / np.sqrt(AnomaliesGF['ret_number'])

#Times 12 to annualize it
AnomaliesGF['ret'] = AnomaliesGF['Mean'] * 12 

#Times Squareroot 12 to annualize it
AnomaliesGF['sd'] = AnomaliesGF['SD'] * np.sqrt(12) 

#Calculate the t-stat by dividing the return by the standard error of the mean. Divide by 12 to
#annualize it
AnomaliesGF['tstat'] = AnomaliesGF['ret'] / AnomaliesGF['ret_semean'] /12

#Format Table
AnomaliesGF[['Mean']] = AnomaliesGF[['ret']].apply(lambda x: x.map("{:.1f}%".format))
AnomaliesGF[['SD']] = AnomaliesGF[['sd']].apply(lambda x: x.map("{:.1f}%".format))
AnomaliesGF[['T-value']] = AnomaliesGF[['tstat']].apply(lambda x: x.map("{:.2f}".format))

print(AnomaliesGF[['anomaly','Mean','SD','T-value']])


  anomaly  Mean     SD T-value
0   glbab  9.6%   9.7%    5.70
1   glcma  1.9%   6.0%    1.74
2   glhml  4.0%   7.4%    2.92
3   glqmj  6.2%   6.8%    5.06
4   glrmw  4.3%   4.7%    4.91
5   glsmb  1.1%   7.1%    0.83
6   glumd  7.9%  12.1%    3.54


In [256]:
#Dit zijn beide US en Global factors combined

AnomaliesUScomb = AnomaliesUS[['anomaly', 'Mean', 'SD', 'T-value']]
AnomaliesGFcomb = AnomaliesGF[['anomaly', 'Mean', 'SD', 'T-value']]

Anomaliescombinedtable = pd.concat([AnomaliesUScomb, AnomaliesGFcomb], ignore_index=True)
print(Anomaliescombinedtable)


   anomaly  Mean     SD T-value
0       ac  2.8%   6.6%    3.19
1      bab  9.8%  11.2%    6.55
2      cfp  3.4%   8.6%    2.94
3      cma  3.3%   6.9%    3.59
4       ep  3.5%   8.9%    2.95
5      hml  3.6%   9.7%    2.82
6      liq  4.4%  11.6%    2.77
7    ltrev  2.5%   8.7%    2.16
8      nsi  2.8%   8.2%    2.52
9      qmj  4.6%   7.7%    4.47
10     rmw  3.1%   7.5%    3.13
11    rvar  1.6%  17.3%    0.68
12     smb  2.7%  10.4%    1.97
13   strev  6.0%  10.6%    4.21
14     umd  7.8%  14.5%    4.02
15   glbab  9.6%   9.7%    5.70
16   glcma  1.9%   6.0%    1.74
17   glhml  4.0%   7.4%    2.92
18   glqmj  6.2%   6.8%    5.06
19   glrmw  4.3%   4.7%    4.91
20   glsmb  1.1%   7.1%    0.83
21   glumd  7.9%  12.1%    3.54


# recreating table 2 of the paper

In [38]:
file_url = 'https://raw.githubusercontent.com/Sebasleen/Seminargroup/Seminar/US%20Factors/anomalies.dta'

# Read the Stata file content
Table2_df = pd.read_stata(file_url)





In [45]:
Table2_df.tail()


,year,month,anomaly,ret,time,global
combined_date,,,,,,
2019-08-01,2019,8,glumd,2.99,715.0,1.0
2019-09-01,2019,9,glumd,-3.26,716.0,1.0
2019-10-01,2019,10,glumd,-0.94,717.0,1.0
2019-11-01,2019,11,glumd,0.00,718.0,1.0
2019-12-01,2019,12,glumd,0.74,719.0,1.0


In [40]:
Table2_df['combined_date'] = pd.to_datetime(Table2_df['year'].astype(str) + '-' + Table2_df['month'].astype(str), format='%Y-%m')

# Optional: If you want to keep 'combined_date' as the first column
Table2_df = Table2_df[['combined_date'] + [col for col in Table2_df.columns if col != 'combined_date']]

In [33]:
Table2_df = Table2_df.drop(['year', 'month'], axis=1)

In [41]:

Table2_df.set_index('combined_date', inplace=True)

In [46]:
# Create a new variable 'Pan' based on unique values of 'anomaly'
Table2_df['Pan'] = Table2_df.groupby('anomaly').grouper.group_info[0]

# Display the modified DataFrame
print(Table2_df)

               year  month anomaly    ret   time  global  Pan
combined_date                                                
1963-07-01     1963      7      ac  2.170   42.0     0.0    0
1963-08-01     1963      8      ac -0.197   43.0     0.0    0
1963-09-01     1963      9      ac  0.600   44.0     0.0    0
1963-10-01     1963     10      ac  6.463   45.0     0.0    0
1963-11-01     1963     11      ac -2.260   46.0     0.0    0
...             ...    ...     ...    ...    ...     ...  ...
2019-08-01     2019      8   glumd  2.990  715.0     1.0   11
2019-09-01     2019      9   glumd -3.260  716.0     1.0   11
2019-10-01     2019     10   glumd -0.940  717.0     1.0   11
2019-11-01     2019     11   glumd  0.000  718.0     1.0   11
2019-12-01     2019     12   glumd  0.740  719.0     1.0   11

[12643 rows x 7 columns]


In [47]:
unique_values_count = Table2_df['Pan'].nunique()
print("Number of unique values in 'Pan':", unique_values_count)

Number of unique values in 'Pan': 22


In [57]:
# Set 'Pan' and 'time' as a MultiIndex
Table2_df.set_index(['Pan', 'time'], inplace=True)

# Generate a variable 'n' representing the time index within each panel
Table2_df['n'] = Table2_df.groupby('Pan').cumcount() + 1


# Display the modified DataFrame
print(Table2_df)

       Pan   time  year  month anomaly    ret  global    n        MA  flag
0        0   54.0  1964      7      ac -0.080     0.0    1  0.959667     1
1        0   55.0  1964      8      ac  0.373     0.0    2  1.007167     1
2        0   56.0  1964      9      ac -0.950     0.0    3  0.878000     1
3        0   57.0  1964     10      ac  0.637     0.0    4  0.392500     1
4        0   58.0  1964     11      ac -0.957     0.0    5  0.501083     1
...    ...    ...   ...    ...     ...    ...     ...  ...       ...   ...
12374   11  715.0  2019      8   glumd  2.990     1.0  334  0.531667     1
12375   11  716.0  2019      9   glumd -3.260     1.0  335  0.158333     1
12376   11  717.0  2019     10   glumd -0.940     1.0  336  0.268333     1
12377   11  718.0  2019     11   glumd  0.000     1.0  337  0.425833     1
12378   11  719.0  2019     12   glumd  0.740     1.0  338  0.320833     1

[12379 rows x 10 columns]


/var/folders/hx/jlx4phmj419bwmhwf_bz4qqm0000gn/T/ipykernel_11568/1375383843.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Table2_df['n'] = Table2_df.groupby('Pan').cumcount() + 1


In [49]:
# Calculate a rolling mean of 'ret' with a window size of 12
Table2_df['MA'] = Table2_df.groupby('Pan')['ret'].transform(lambda x: x.rolling(window=12, min_periods=1).mean())

# Drop observations where 'n' is less than 13
Table2_df = Table2_df[Table2_df['n'] >= 13]


# Display the modified DataFrame
print(Table2_df)

       index  Pan   time  year  month anomaly    ret  global    n        MA
0         12    0   54.0  1964      7      ac -0.080     0.0   13  0.959667
1         13    0   55.0  1964      8      ac  0.373     0.0   14  1.007167
2         14    0   56.0  1964      9      ac -0.950     0.0   15  0.878000
3         15    0   57.0  1964     10      ac  0.637     0.0   16  0.392500
4         16    0   58.0  1964     11      ac -0.957     0.0   17  0.501083
...      ...  ...    ...   ...    ...     ...    ...     ...  ...       ...
12374  12638   11  715.0  2019      8   glumd  2.990     1.0  346  0.531667
12375  12639   11  716.0  2019      9   glumd -3.260     1.0  347  0.158333
12376  12640   11  717.0  2019     10   glumd -0.940     1.0  348  0.268333
12377  12641   11  718.0  2019     11   glumd  0.000     1.0  349  0.425833
12378  12642   11  719.0  2019     12   glumd  0.740     1.0  350  0.320833

[12379 rows x 10 columns]


In [56]:

Table2_df['flag'] = Table2_df['MA'].apply(lambda x: 1 if x > 0 else 0)

# Replace negative values in 'flag' with 0
Table2_df['flag'] = Table2_df['flag'].replace(-1, 0)

# Display the modified DataFrame
print(Table2_df)

       Pan   time  year  month anomaly    ret  global    n        MA  flag
0        0   54.0  1964      7      ac -0.080     0.0   13  0.959667     1
1        0   55.0  1964      8      ac  0.373     0.0   14  1.007167     1
2        0   56.0  1964      9      ac -0.950     0.0   15  0.878000     1
3        0   57.0  1964     10      ac  0.637     0.0   16  0.392500     1
4        0   58.0  1964     11      ac -0.957     0.0   17  0.501083     1
...    ...    ...   ...    ...     ...    ...     ...  ...       ...   ...
12374   11  715.0  2019      8   glumd  2.990     1.0  346  0.531667     1
12375   11  716.0  2019      9   glumd -3.260     1.0  347  0.158333     1
12376   11  717.0  2019     10   glumd -0.940     1.0  348  0.268333     1
12377   11  718.0  2019     11   glumd  0.000     1.0  349  0.425833     1
12378   11  719.0  2019     12   glumd  0.740     1.0  350  0.320833     1

[12379 rows x 10 columns]


/var/folders/hx/jlx4phmj419bwmhwf_bz4qqm0000gn/T/ipykernel_11568/1284322700.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Table2_df['flag'] = Table2_df['MA'].apply(lambda x: 1 if x > 0 else 0)
/var/folders/hx/jlx4phmj419bwmhwf_bz4qqm0000gn/T/ipykernel_11568/1284322700.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Table2_df['flag'] = Table2_df['flag'].replace(-1, 0)


In [58]:
# List of variables to generate
variables_to_generate = ["a0", "a0t", "b0", "b0t", "R20", "a1", "a1t", "b1", "b1t", "R21"]

# Generate variables with NaN values
for v in variables_to_generate:
    Table2_df[v] = float('nan')

# Display the DataFrame with the generated variables
print(Table2_df)

       Pan   time  year  month anomaly    ret  global    n        MA  flag  \
0        0   54.0  1964      7      ac -0.080     0.0    1  0.959667     1   
1        0   55.0  1964      8      ac  0.373     0.0    2  1.007167     1   
2        0   56.0  1964      9      ac -0.950     0.0    3  0.878000     1   
3        0   57.0  1964     10      ac  0.637     0.0    4  0.392500     1   
4        0   58.0  1964     11      ac -0.957     0.0    5  0.501083     1   
...    ...    ...   ...    ...     ...    ...     ...  ...       ...   ...   
12374   11  715.0  2019      8   glumd  2.990     1.0  334  0.531667     1   
12375   11  716.0  2019      9   glumd -3.260     1.0  335  0.158333     1   
12376   11  717.0  2019     10   glumd -0.940     1.0  336  0.268333     1   
12377   11  718.0  2019     11   glumd  0.000     1.0  337  0.425833     1   
12378   11  719.0  2019     12   glumd  0.740     1.0  338  0.320833     1   

       a0  a0t  b0  b0t  R20  a1  a1t  b1  b1t  R21  
0     NaN

/var/folders/hx/jlx4phmj419bwmhwf_bz4qqm0000gn/T/ipykernel_11568/1497670246.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Table2_df[v] = float('nan')
/var/folders/hx/jlx4phmj419bwmhwf_bz4qqm0000gn/T/ipykernel_11568/1497670246.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Table2_df[v] = float('nan')
/var/folders/hx/jlx4phmj419bwmhwf_bz4qqm0000gn/T/ipykernel_11568/1497670246.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [59]:
# Step 1: Duplicate the DataFrame
temp_df = Table2_df.copy()

# Step 2: Calculate summary statistics for the variable 'Pan'
max_pan = temp_df['Pan'].max()

# Step 3: Increase the maximum value of 'Pan' by 1
temp_df['Pan'] = max_pan + 1

# Step 4: Replace the values in the 'anomaly' variable with the string 'Everything'
temp_df['anomaly'] = 'Everything'



# Display the modified DataFrame
print(Table2_df)

       Pan   time  year  month     anomaly    ret  global    n        MA  \
0        0   54.0  1964      7          ac -0.080     0.0    1  0.959667   
1        0   55.0  1964      8          ac  0.373     0.0    2  1.007167   
2        0   56.0  1964      9          ac -0.950     0.0    3  0.878000   
3        0   57.0  1964     10          ac  0.637     0.0    4  0.392500   
4        0   58.0  1964     11          ac -0.957     0.0    5  0.501083   
...    ...    ...   ...    ...         ...    ...     ...  ...       ...   
24753   22  715.0  2019      8  Everything  2.990     1.0  334  0.531667   
24754   22  716.0  2019      9  Everything -3.260     1.0  335  0.158333   
24755   22  717.0  2019     10  Everything -0.940     1.0  336  0.268333   
24756   22  718.0  2019     11  Everything  0.000     1.0  337  0.425833   
24757   22  719.0  2019     12  Everything  0.740     1.0  338  0.320833   

       flag  a0  a0t  b0  b0t  R20  a1  a1t  b1  b1t  R21  
0         1 NaN  NaN NaN  N

In [60]:
temp_df.head()

,Pan,time,year,month,anomaly,ret,global,n,MA,flag,a0,a0t,b0,b0t,R20,a1,a1t,b1,b1t,R21
0,22,54.0,1964,7,Everything,-0.080,0.0,1,0.959667,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,22,55.0,1964,8,Everything,0.373,0.0,2,1.007167,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,22,56.0,1964,9,Everything,-0.950,0.0,3,0.878000,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,22,57.0,1964,10,Everything,0.637,0.0,4,0.392500,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,22,58.0,1964,11,Everything,-0.957,0.0,5,0.501083,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
# Perform Pooled Regressions
for i in range(1, Table2_df['Pan'].max() + 1):
    # Spec 1: x = average return
    if i < Table2_df['Pan'].max() + 1:
        model_spec_1 = sm.OLS(Table2_df.loc[Table2_df['Pan'] == i, 'ret'], sm.add_constant(Table2_df.loc[Table2_df['Pan'] == i, 'MA']))
    else:
        model_spec_1 = sm.OLS(Table2_df.loc[Table2_df['Pan'] == i, 'ret'], sm.add_constant(Table2_df.loc[Table2_df['Pan'] == i, 'MA'])).fit(cov_type='cluster', cov_kwds={'groups': Table2_df.loc[Table2_df['Pan'] == i, 'time']})

    Table2_df.loc[Table2_df['Pan'] == i, 'a0'] = model_spec_1.params['const']
    Table2_df.loc[Table2_df['Pan'] == i, 'a0t'] = model_spec_1.tvalues['const']
    Table2_df.loc[Table2_df['Pan'] == i, 'b0'] = model_spec_1.params['MA']
    Table2_df.loc[Table2_df['Pan'] == i, 'b0t'] = model_spec_1.tvalues['MA']

    # Spec 2: x = average return > 0
    if i < Table2_df['Pan'].max() + 1:
        model_spec_2 = sm.OLS(Table2_df.loc[Table2_df['Pan'] == i, 'ret'], sm.add_constant(Table2_df.loc[Table2_df['Pan'] == i, 'flag']))
    else:
        model_spec_2 = sm.OLS(Table2_df.loc[Table2_df['Pan'] == i, 'ret'], sm.add_constant(Table2_df.loc[Table2_df['Pan'] == i, 'flag'])).fit(cov_type='cluster', cov_kwds={'groups': Table2_df.loc[Table2_df['Pan'] == i, 'time']})

    Table2_df.loc[Table2_df['Pan'] == i, 'a1'] = model_spec_2.params['const']
    Table2_df.loc[Table2_df['Pan'] == i, 'a1t'] = model_spec_2.tvalues['const']
    Table2_df.loc[Table2_df['Pan'] == i, 'b1'] = model_spec_2.params['flag']
    Table2_df.loc[Table2_df['Pan'] == i, 'b1t'] = model_spec_2.tvalues['flag']

# Display the modified DataFrame
print(Table2_df)

AttributeError: 'OLS' object has no attribute 'params'

In [62]:
# Perform Pooled Regressions
for i in range(1, Table2_df['Pan'].max() + 1):
    # Spec 1: x = average return
    if i < Table2_df['Pan'].max() + 1:
        model_spec_1 = sm.OLS(Table2_df.loc[Table2_df['Pan'] == i, 'ret'], sm.add_constant(Table2_df.loc[Table2_df['Pan'] == i, 'MA'])).fit()
    else:
        model_spec_1 = sm.OLS(Table2_df.loc[Table2_df['Pan'] == i, 'ret'], sm.add_constant(Table2_df.loc[Table2_df['Pan'] == i, 'MA'])).fit(cov_type='cluster', cov_kwds={'groups': Table2_df.loc[Table2_df['Pan'] == i, 'time']})

    Table2_df.loc[Table2_df['Pan'] == i, 'a0'] = model_spec_1.params['const']
    Table2_df.loc[Table2_df['Pan'] == i, 'a0t'] = model_spec_1.tvalues['const']
    Table2_df.loc[Table2_df['Pan'] == i, 'b0'] = model_spec_1.params['MA']
    Table2_df.loc[Table2_df['Pan'] == i, 'b0t'] = model_spec_1.tvalues['MA']

    # Spec 2: x = average return > 0
    if i < Table2_df['Pan'].max() + 1:
        model_spec_2 = sm.OLS(Table2_df.loc[Table2_df['Pan'] == i, 'ret'], sm.add_constant(Table2_df.loc[Table2_df['Pan'] == i, 'flag'])).fit()
    else:
        model_spec_2 = sm.OLS(Table2_df.loc[Table2_df['Pan'] == i, 'ret'], sm.add_constant(Table2_df.loc[Table2_df['Pan'] == i, 'flag'])).fit(cov_type='cluster', cov_kwds={'groups': Table2_df.loc[Table2_df['Pan'] == i, 'time']})

    Table2_df.loc[Table2_df['Pan'] == i, 'a1'] = model_spec_2.params['const']
    Table2_df.loc[Table2_df['Pan'] == i, 'a1t'] = model_spec_2.tvalues['const']
    Table2_df.loc[Table2_df['Pan'] == i, 'b1'] = model_spec_2.params['flag']
    Table2_df.loc[Table2_df['Pan'] == i, 'b1t'] = model_spec_2.tvalues['flag']

# Display the modified DataFrame
print(Table2_df)

       Pan   time  year  month     anomaly    ret  global    n        MA  \
0        0   54.0  1964      7          ac -0.080     0.0    1  0.959667   
1        0   55.0  1964      8          ac  0.373     0.0    2  1.007167   
2        0   56.0  1964      9          ac -0.950     0.0    3  0.878000   
3        0   57.0  1964     10          ac  0.637     0.0    4  0.392500   
4        0   58.0  1964     11          ac -0.957     0.0    5  0.501083   
...    ...    ...   ...    ...         ...    ...     ...  ...       ...   
24753   22  715.0  2019      8  Everything  2.990     1.0  334  0.531667   
24754   22  716.0  2019      9  Everything -3.260     1.0  335  0.158333   
24755   22  717.0  2019     10  Everything -0.940     1.0  336  0.268333   
24756   22  718.0  2019     11  Everything  0.000     1.0  337  0.425833   
24757   22  719.0  2019     12  Everything  0.740     1.0  338  0.320833   

       flag        a0       a0t        b0        b0t  R20        a1  \
0         1     

In [63]:
variables_to_collapse = ["a0", "a0t", "b0", "b0t", "a1", "a1t", "b1", "b1t"]

# Perform the collapse operation
collapsed_df = Table2_df.groupby(['global', 'Pan', 'anomaly'])[variables_to_collapse].mean().reset_index()

# Display the collapsed DataFrame
print(collapsed_df)

    global  Pan     anomaly        a0       a0t        b0        b0t  \
0      0.0    0          ac       NaN       NaN       NaN        NaN   
1      0.0    1         bab  0.033340  0.244592  0.955583  11.187071   
2      0.0    2         cfp  0.022559  0.229649  0.887695   7.996280   
3      0.0    3         cma  0.006223  0.078778  0.967900   9.047547   
4      0.0    4          ep  0.034666  0.346296  0.869458   8.292976   
5      0.0   12         hml  0.014287  0.131481  0.910260   8.673239   
6      0.0   13         liq -0.053629 -0.387293  1.118074   8.474218   
7      0.0   14       ltrev -0.007191 -0.076016  0.980277   9.343956   
8      0.0   15         nsi -0.015455 -0.173557  1.042971  10.229237   
9      0.0   16         qmj  0.010406  0.112843  0.971691   9.067753   
10     0.0   17         rmw  0.015070  0.180126  0.964880   9.262643   
11     0.0   18        rvar -0.004337 -0.023491  1.018337   8.908651   
12     0.0   19         smb  0.000556  0.004836  0.995961   8.56